# Using ML Anonymization to Defend Against Membership Inference Attacks
In this tutorial, we will show how to anonymize models using the ML anonymization module.

We will demonstrate running inference attacks both on a vanilla model and on an anonymized version of the model. We will run a black-box membership inference attack using ART's inference module.

Dataset used: [Adult dataset](https://archive.ics.uci.edu/ml/datasets/adult).

For simplicity, only numerical features are used.

In [ ]:
!mkdir -p ../datasets
!wget -P ../datasets https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
!wget -P ../datasets https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data
!wget -P ../datasets https://archive.ics.uci.edu/ml/machine-learning-databases/nursery/nursery.data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
data = pd.read_csv('../datasets/adult.data', header=None, skipinitialspace=True)
num_features = [0, 4, 10, 11, 12]
X = data.iloc[:, num_features].values
y = data.iloc[:, 14].values

# Cleaning labels
y = np.array([label.strip() for label in y])
y = np.where(y == '<=50K', 0, 1)

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

print(f'Final Train Set: {x_train.shape}, Test Set: {x_test.shape}')

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from art.estimators.classification.scikitlearn import ScikitlearnDecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x_train, y_train)
art_classifier = ScikitlearnDecisionTreeClassifier(model)

print('Base model accuracy: ', model.score(x_test, y_test))

In [ ]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox

bb_attack = MembershipInferenceBlackBox(art_classifier, attack_model_type='rf')

attack_train_ratio = 0.5
attack_train_size = int(len(x_train) * attack_train_ratio)
attack_test_size = int(len(x_test) * attack_train_ratio)

bb_attack.fit(x_train[:attack_train_size], y_train[:attack_train_size],
              x_test[:attack_test_size], y_test[:attack_test_size])

inferred_train_bb = bb_attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
inferred_test_bb = bb_attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])

train_acc = np.sum(inferred_train_bb) / len(inferred_train_bb)
test_acc = 1 - (np.sum(inferred_test_bb) / len(inferred_test_bb))

acc = (train_acc * len(inferred_train_bb) + test_acc * len(inferred_test_bb)) / (len(inferred_train_bb) + len(inferred_test_bb))

print(acc)

In [ ]:
!pip install ai-privacy-toolkit

In [ ]:
from apt.utils.datasets import ArrayDataset
from apt.anonymization import Anonymize

QI = [0, 1, 2, 4]
anonymizer = Anonymize(100, QI)
anon = anonymizer.anonymize(ArrayDataset(x_train, np.array([np.argmax(arr) for arr in art_classifier.predict(x_train)])))

print(anon)

anon_model = DecisionTreeClassifier()
anon_model.fit(anon, y_train)
anon_art_classifier = ScikitlearnDecisionTreeClassifier(anon_model)

print('Anonymized model accuracy: ', anon_model.score(x_test, y_test))

In [ ]:
anon_bb_attack = MembershipInferenceBlackBox(anon_art_classifier, attack_model_type='rf')

anon_bb_attack.fit(x_train[:attack_train_size], y_train[:attack_train_size],
                   x_test[:attack_test_size], y_test[:attack_test_size])

anon_inferred_train_bb = anon_bb_attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
anon_inferred_test_bb = anon_bb_attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])

anon_train_acc = np.sum(anon_inferred_train_bb) / len(anon_inferred_train_bb)
anon_test_acc = 1 - (np.sum(anon_inferred_test_bb) / len(anon_inferred_test_bb))

anon_acc = (anon_train_acc * len(anon_inferred_train_bb) + anon_test_acc * len(anon_inferred_test_bb)) / (len(anon_inferred_train_bb) + len(anon_inferred_test_bb))

print(anon_acc)

In [ ]:
def calc_precision_recall(predicted, actual, positive_value=1):
    score = 0
    num_positive_predicted = sum(predicted == positive_value)
    num_positive_actual = sum(actual == positive_value)
    for i in range(len(predicted)):
        if predicted[i] == actual[i] and predicted[i] == positive_value:
            score += 1
    precision = score / num_positive_predicted if num_positive_predicted else 1
    recall = score / num_positive_actual if num_positive_actual else 1
    return precision, recall

print('without anonymization:', calc_precision_recall(np.concatenate((inferred_train_bb, inferred_test_bb)),
                        np.concatenate((np.ones(len(inferred_train_bb)), np.zeros(len(inferred_test_bb))))))

print('with anonymization:', calc_precision_recall(np.concatenate((anon_inferred_train_bb, anon_inferred_test_bb)),
                        np.concatenate((np.ones(len(anon_inferred_train_bb)), np.zeros(len(anon_inferred_test_bb))))))